In [9]:
import matplotlib.pyplot as plt
from Objects.Simulation import Simulation
import numpy as np
import pandas as pd
%pylab
%matplotlib inline



Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [10]:
%pylab
%matplotlib inline

from matplotlib import lines, markers
from cycler import cycler

# Create cycler object. Use any styling from above you please
monochromeWithMarker = (cycler('color', ['k']) * cycler('linestyle', ['-', '--', ':', '-.']) * cycler('marker', [ '.' ,'^']))
monochromeNoMarker = (cycler('color', ['k']) * cycler('linestyle', ['-', '--', ':', '-.']) * cycler('marker', [',']))
monochrome = monochromeNoMarker.concat(monochromeWithMarker)

plt.set_cmap('gray')
# Overriding styles for current script
plt.rcParams['axes.grid'] = True
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.bottom'] = False
plt.rcParams['axes.spines.left'] = False
plt.rcParams['figure.figsize'] = (12,5)
mpl.rcParams['figure.dpi']= 300
plt.rcParams['axes.prop_cycle'] = monochrome

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


<Figure size 432x288 with 0 Axes>

In [11]:
def cal_one_p(p):
    return np.mean(p.delays), np.max(p.delays)

def cal_vdr(ps, tg):
    if type(ps) is not list:
        ps = [ps]
    total_vehPass = 0
    total_tg = 0
    for p in ps:
        total_vehPass += p.vehPass
        total_tg += tg
    return total_vehPass / total_tg * 3600

def get_data_array(p,tg):
    delaymean, delaymax = cal_one_p(p)
    return [delaymean, delaymax, cal_vdr(p, tg)]

In [12]:
simulation_time = 60 # seconds
n = 100 # number of vehicles
intended_speed = 80 * 1000/3600 # intended speed / speed limit of the road
sim = Simulation(time = simulation_time, avStep = 100)


LRT = [1.34, -1.34]
HSR = [0.58, -0.54]

human_profile = [1.7, -2.0]
av_profile = HSR

paras={
    'v_intend': intended_speed,
    'human_max_acc': human_profile[0],
    'human_max_dec': human_profile[1],
    'max_acc': av_profile[0],
    'max_dec': av_profile[1],
    'tsys': 1.0,
    'human_T': 1.8,
    'AV_T': 1.6
}

av = sim.run_av_simulation(n, paras)
get_data_array(av, simulation_time)

[0.12942767311532427, 0.2537603430021795, 1740.0]

In [13]:
human_ps = []
av_ps = []
cav_ps = []
start_acc = 1.34
end_acc = 0.58
start_dec = -1.34
end_dec = -0.54
totalLoop = int((start_acc - end_acc-0.0001) / 0.04 + 1) + 1
secondLoop = int((end_dec - start_dec -0.0001) / 0.04 + 1) + 1
print(totalLoop, secondLoop)
xs_acc = [round(x,2) for x in [start_acc - i * 0.04 for i in range(totalLoop)]]
ys_dec = [round(x,2) for x in [start_dec + j * 0.04 for j in range(secondLoop)]]

20 21


In [14]:
av_avg_delay = pd.DataFrame(index = ys_dec, columns=xs_acc)
av_max_delay = pd.DataFrame(index = ys_dec, columns=xs_acc)
av_vdr = pd.DataFrame(index = ys_dec, columns=xs_acc)
cav_avg_delay = pd.DataFrame(index = ys_dec, columns=xs_acc)
cav_max_delay = pd.DataFrame(index = ys_dec, columns=xs_acc)
cav_vdr = pd.DataFrame(index = ys_dec, columns=xs_acc)


for col in xs_acc:
    cur_acc = col
    for idx in ys_dec:
        cur_dec = idx
        paras['max_acc'] = cur_acc
        paras['max_dec'] = cur_dec
        print(cur_acc,cur_dec)
        av = sim.run_av_simulation(n, paras)
        cav = sim.run_cav_simulation(n, paras)
        av=get_data_array(av, simulation_time)
        cav=get_data_array(cav, simulation_time)
        print(cav)
        av_avg_delay.at[idx, col] = av[0]
        av_max_delay.at[idx, col] = av[1]
        av_vdr.at[idx, col] = av[2]
        cav_avg_delay.at[idx, col] = cav[0]
        cav_max_delay.at[idx, col] = cav[1]
        cav_vdr.at[idx, col] = cav[2]

1.34 -1.34
[0.0481019658793009, 0.1130553813332567, 3180.0]
1.34 -1.3
[0.04810188971361401, 0.11305463887245983, 3180.0]
1.34 -1.26
[0.0481018103978366, 0.11305387415146072, 3180.0]
1.34 -1.22
[0.04810172563126293, 0.113053065905998, 3180.0]
1.34 -1.18
[0.04810163314866325, 0.11305220688364502, 3180.0]
1.34 -1.14
[0.04810153663375286, 0.1130513191587996, 3180.0]
1.34 -1.1
[0.04810143224830993, 0.1130503798668617, 3180.0]
1.34 -1.06
[0.04810131866462747, 0.11304937399855587, 3180.0]
1.34 -1.02
[0.048101197156042254, 0.11304830858648864, 3180.0]
1.34 -0.98
[0.04810106552957804, 0.11304717223577311, 3180.0]
1.34 -0.94
[0.04810092089433624, 0.11304594197010151, 3180.0]
1.34 -0.9
[0.04810076480517818, 0.11304461866483033, 3180.0]
1.34 -0.86
[0.04810059280306674, 0.11304318048199917, 3180.0]
1.34 -0.82
[0.04810040482333687, 0.11304161918186542, 3180.0]
1.34 -0.78
[0.04810019790114339, 0.11303991180466672, 3180.0]
1.34 -0.74
[0.04809996846346259, 0.11303803879914895, 3180.0]
1.34 -0.7
[0.0480

[0.05172059862972341, 0.11865195309704447, 3120.0]
1.1 -1.02
[0.05172052001593597, 0.11865120240872018, 3120.0]
1.1 -0.98
[0.051720435671582576, 0.118650408923446, 3120.0]
1.1 -0.94
[0.05172034274780118, 0.1186495550474875, 3120.0]
1.1 -0.9
[0.05172024036124482, 0.1186486261891803, 3120.0]
1.1 -0.86
[0.05172012815522097, 0.11864762740825559, 3120.0]
1.1 -0.82
[0.05172000329587996, 0.11864653461674583, 3120.0]
1.1 -0.78
[0.05171986439397827, 0.11864533417355295, 3120.0]
1.1 -0.74
[0.051719711172835504, 0.11864402543466553, 3120.0]
1.1 -0.7
[0.05171953781578861, 0.11864256535991, 3120.0]
1.1 -0.66
[0.05171934549904748, 0.11864095593114447, 3120.0]
1.1 -0.62
[0.05171912792586006, 0.11863915538846388, 3120.0]
1.1 -0.58
[0.05171888175258528, 0.11863713610020227, 3120.0]
1.1 -0.54
[0.05171860054293263, 0.11863485112124152, 3120.0]
1.06 -1.34
[0.05279802648223809, 0.12066728316075646, 3120.0]
1.06 -1.3
[0.0527979811331744, 0.1206667735456722, 3120.0]
1.06 -1.26
[0.05279793339205696, 0.1206662

[0.057227016086975356, 0.12606388370456822, 3000.0]
0.86 -0.7
[0.05722692589934535, 0.12606304432912488, 3000.0]
0.86 -0.66
[0.0572268211231117, 0.12606208963136165, 3000.0]
0.86 -0.62
[0.057226702059051435, 0.12606102619746812, 3000.0]
0.86 -0.58
[0.05722656391931097, 0.12605981253888926, 3000.0]
0.86 -0.54
[0.05722640447216659, 0.12605844136000144, 3000.0]
0.82 -1.34
[0.05917184822165906, 0.1295430405718377, 3000.0]
0.82 -1.3
[0.0591718321259948, 0.1295428139569365, 3000.0]
0.82 -1.26
[0.05917181406538843, 0.12954256549970694, 3000.0]
0.82 -1.22
[0.05917179411431865, 0.12954229939687234, 3000.0]
0.82 -1.18
[0.059171771950630905, 0.12954201078040345, 3000.0]
0.82 -1.14
[0.05917174759709803, 0.12954170092455614, 3000.0]
0.82 -1.1
[0.059171720984374995, 0.12954137006503696, 3000.0]
0.82 -1.06
[0.05917169080344935, 0.129541006991087, 3000.0]
0.82 -1.02
[0.05917165806801164, 0.12954062317689302, 3000.0]
0.82 -0.98
[0.05917162171663838, 0.12954021016986053, 3000.0]
0.82 -0.94
[0.0591715807

[0.07183924863231464, 0.14479147403400325, 2760.0]
0.58 -1.22
[0.07183924769907842, 0.14479145227829462, 2760.0]
0.58 -1.18
[0.07183924637643582, 0.14479142369208525, 2760.0]
0.58 -1.14
[0.07183924477977849, 0.14479139000899377, 2760.0]
0.58 -1.1
[0.07183924260223863, 0.1447913468713701, 2760.0]
0.58 -1.06
[0.07183923991142413, 0.14479129553184286, 2760.0]
0.58 -1.02
[0.07183923656389211, 0.1447912349274531, 2760.0]
0.58 -0.98
[0.07183923227111524, 0.1447911611050954, 2760.0]
0.58 -0.94
[0.07183922705656752, 0.14479107473819575, 2760.0]
0.58 -0.9
[0.07183922054719202, 0.1447909716693908, 2760.0]
0.58 -0.86
[0.07183921248259945, 0.144790850638792, 2760.0]
0.58 -0.82
[0.07183920270805717, 0.14479071022032175, 2760.0]
0.58 -0.78
[0.07183919034569118, 0.14479054178512143, 2760.0]
0.58 -0.74
[0.07183917558584382, 0.14479034748974204, 2760.0]
0.58 -0.7
[0.07183915652524597, 0.14479010969547979, 2760.0]
0.58 -0.66
[0.0718391331073706, 0.144789829309581, 2760.0]
0.58 -0.62
[0.07183910419103469

In [15]:
av_avg_delay * 1000

,1.34,1.3,1.26,1.22,1.18,1.14,1.1,1.06,1.02,0.98,0.94,0.9,0.86,0.82,0.78,0.74,0.7,0.66,0.62,0.58
-1.34,92.4196,90.3888,92.4703,94.7085,97.1148,99.7011,98.2372,101.191,104.375,107.808,107.097,111.076,115.382,115.511,120.573,126.077,127.464,134.046,141.25,144.582
-1.30,91.7889,93.7592,91.8318,94.0558,96.449,99.0236,97.564,100.507,103.681,107.107,106.398,110.371,114.673,114.805,119.866,125.372,131.368,133.352,140.567,143.907
-1.26,91.1355,93.0906,91.1699,93.3786,95.7576,98.3194,96.8645,99.7947,102.959,106.376,105.67,109.636,113.934,118.593,119.128,124.636,130.637,132.627,139.853,147.782
-1.22,90.458,92.3968,90.4833,92.6754,95.0389,97.5867,100.332,99.0537,102.206,105.613,104.911,108.869,113.162,117.819,118.357,123.866,129.873,131.869,139.105,147.051
-1.18,89.7554,91.6763,93.744,91.9448,94.2915,96.8237,99.5554,98.2816,101.421,104.818,108.491,108.068,112.355,117.009,117.551,123.061,129.073,131.074,138.322,146.284
-1.14,89.0261,90.9277,92.977,91.1851,93.5134,96.0287,98.7451,97.4764,100.602,103.986,107.65,107.231,111.51,116.16,116.707,122.217,128.233,130.241,137.499,145.479
-1.10,88.2687,90.1494,92.1786,94.3675,92.7027,95.1994,97.8989,100.817,99.7453,103.116,106.769,106.354,110.625,115.27,115.822,121.331,127.351,129.366,136.635,144.632
-1.06,87.4819,89.3396,91.3468,93.515,91.8576,94.3336,97.0144,99.9156,98.8492,102.205,105.846,105.435,109.696,114.335,119.387,120.399,126.422,128.446,135.725,143.739
-1.02,86.6638,88.4966,90.4798,92.625,94.9447,93.429,96.0889,98.9715,97.9107,101.249,104.876,108.815,108.72,113.352,118.4,119.419,125.444,132.045,134.765,142.796
-0.98,89.5423,87.6186,89.5753,91.6953,93.9912,92.4829,95.1196,97.9812,101.085,100.246,103.857,107.783,107.693,112.316,117.36,118.384,124.412,131.02,133.751,141.8


In [16]:
subFolder = "SG1/"
av_avg_delay.to_csv(subFolder+'SG1_av_avg_delay.csv')
av_max_delay.to_csv(subFolder+'SG1_av_max_delay.csv')
av_vdr.to_csv(subFolder+'SG1_av_vdr.csv')
cav_avg_delay.to_csv(subFolder+'SG1_cav_avg_delay.csv')
cav_max_delay.to_csv(subFolder+'SG1_cav_max_delay.csv')
cav_vdr.to_csv(subFolder+'SG1_cav_vdr.csv')